In [1]:
#環境がcolabの場合
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#f1=open('drive/My Drive/call.txt')
#f2=open('drive/My Drive/reply.txt')
f1=open('call.txt')
f2=open('reply.txt')
call_limited=[]
reply_limited=[]
for line1,line2 in zip(f1,f2):
    if (len(line2) <= 50) and (len(line1) <= 50):
        if ("…" not in line1) and ("…" not in line2):
            call_limited.append(line1)
            reply_limited.append(line2)

In [2]:
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

def load_data2(arr):
    tokenizer = Tokenizer(filters="")
    whole_texts = []
    count=0
    for line in arr:
        whole_texts.append("<s> " + line.strip() + " </s>")
    tokenizer.fit_on_texts(whole_texts)

    return tokenizer.texts_to_sequences(whole_texts), tokenizer

x_train, tokenizer_en = load_data2(call_limited)
y_train, tokenizer_ja = load_data2(reply_limited)

en_vocab_size = len(tokenizer_en.word_index) + 1
ja_vocab_size = len(tokenizer_ja.word_index) + 1

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.02, random_state=42)

# パディング
x_train = pad_sequences(x_train, padding='post')
y_train = pad_sequences(y_train, padding='post')

seqX_len = len(x_train[0])
seqY_len = len(y_train[0])

Using TensorFlow backend.


In [0]:
from keras.models import Model
from keras.layers import Input, Permute, Activation, Embedding, Dense, LSTM, concatenate, dot
from keras import backend as K

emb_dim = 256
hid_dim = 256
att_dim = 256

# 符号化器
encoder_inputs = Input(shape=(seqX_len,))
encoder_embedded = Embedding(en_vocab_size, emb_dim, mask_zero=True)(encoder_inputs)
encoded_seq, *encoder_states = LSTM(hid_dim, return_sequences=True, return_state=True)(encoder_embedded)

# 復号化器（encoder_statesを初期状態として指定）
decoder_inputs = Input(shape=(seqY_len,))
decoder_embedding = Embedding(ja_vocab_size, emb_dim)
decoder_embedded = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(hid_dim, return_sequences=True, return_state=True)
decoded_seq, _, _ = decoder_lstm(decoder_embedded, initial_state=encoder_states)

# Attention
score_dense = Dense(hid_dim)
score = score_dense(decoded_seq)                        # shape: (seqY_len, hid_dim) -> (seqY_len, hid_dim)
score = dot([score, encoded_seq], axes=(2,2))           # shape: [(seqY_len, hid_dim), (seqX_len, hid_dim)] -> (seqY_len, seqX_len)
attention = Activation('softmax')(score)                # shape: (seqY_len, seqX_len) -> (seqY_len, seqX_len)
context = dot([attention, encoded_seq], axes=(2,1))     # shape: [(seqY_len, seqX_len), (seqX_len, hid_dim)] -> (seqY_len, hid_dim)
concat = concatenate([context, decoded_seq], axis=2)    # shape: [(seqY_len, hid_dim), (seqY_len, hid_dim)] -> (seqY_len, 2*hid_dim)
attention_dense = Dense(att_dim, activation='tanh')
attentional = attention_dense(concat)                   # shape: (seqY_len, 2*hid_dim) -> (seqY_len, att_dim)
output_dense = Dense(ja_vocab_size, activation='softmax')
outputs = output_dense(attentional)                     # shape: (seqY_len, att_dim) -> (seqY_len, ja_vocab_size)

model = Model([encoder_inputs, decoder_inputs], outputs)
#model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [6]:
import numpy as np
train_target = np.hstack((y_train[:, 1:], np.zeros((len(y_train),1), dtype=np.int32)))

#cp_cb = ModelCheckpoint(filepath = fpath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
model.fit([x_train, y_train], np.expand_dims(train_target, -1), batch_size=128, epochs=5, verbose=1, validation_split=0.2)

Train on 314528 samples, validate on 78632 samples
Epoch 1/5
314528/314528 [==============================] - 2012s 6ms/step - loss: 1.8961 - val_loss: 1.6303
Epoch 2/5
314528/314528 [==============================] - 2010s 6ms/step - loss: 1.5657 - val_loss: 1.5409
Epoch 3/5
314528/314528 [==============================] - 2011s 6ms/step - loss: 1.4905 - val_loss: 1.4989
Epoch 4/5
314528/314528 [==============================] - 2012s 6ms/step - loss: 1.4371 - val_loss: 1.4781
Epoch 5/5
314528/314528 [==============================] - 2013s 6ms/step - loss: 1.3925 - val_loss: 1.4590


In [11]:
model.save("chatbot_u50_e3.model")

/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [12]:
#import shutil
#shutil.copyfile("/content/chatbot_u50_e3.model", "drive/My Drive/chatbot_u50_e3.model")

'drive/My Drive/chatbot_u50_e3.model'

In [0]:
from keras.models import load_model
#model=load_model("drive/My Drive/chatbot_u50_e3.model")
model.load_weights("drive/My Drive/chatbot_u50_e3.model")
#model.load_weights("drive/My Drive/chatbot_50v2_e5.model")

In [0]:
from keras.models import Model
from keras.layers import Input, Permute, Activation, Embedding, Dense, LSTM, concatenate, dot
from keras import backend as K

In [0]:
encoder_model = Model(encoder_inputs, [encoded_seq]+encoder_states)

decoder_states_inputs = [Input(shape=(hid_dim,)), Input(shape=(hid_dim,))]

decoder_inputs = Input(shape=(1,))
decoder_embedded = decoder_embedding(decoder_inputs)
decoded_seq, *decoder_states = decoder_lstm(decoder_embedded, initial_state=decoder_states_inputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoded_seq] + decoder_states)

# Attention
encoded_seq_in, decoded_seq_in = Input(shape=(seqX_len, hid_dim)), Input(shape=(1, hid_dim))
score = score_dense(decoded_seq_in)
score = dot([score, encoded_seq_in], axes=(2,2))
attention = Activation('softmax')(score)
context = dot([attention, encoded_seq_in], axes=(2,1))
concat = concatenate([context, decoded_seq_in], axis=2)
attentional = attention_dense(concat)
attention_outputs = output_dense(attentional)

attention_model = Model([encoded_seq_in, decoded_seq_in], [attention_outputs, attention])

In [0]:
def decode_sequence(input_seq, bos_eos, max_output_length = 1000):
    encoded_seq, *states_value = encoder_model.predict(input_seq)

    target_seq = np.array(bos_eos[0])  # bos_eos[0]="<s>"に対応するインデックス
    output_seq = bos_eos[0][:]
    attention_seq = np.empty((0,len(input_seq[0])))
    
    while True:
        decoded_seq, *states_value = decoder_model.predict([target_seq] + states_value)
        output_tokens, attention = attention_model.predict([encoded_seq, decoded_seq])
        sampled_token_index = [np.argmax(output_tokens[0, -1, :])]
        output_seq += sampled_token_index
        attention_seq = np.append(attention_seq, attention[0], axis=0)
        
        if (sampled_token_index == bos_eos[1] or len(output_seq) > max_output_length):
            break

        target_seq = np.array(sampled_token_index)

    return output_seq, attention_seq

In [0]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np

detokenizer_en = dict(map(reversed, tokenizer_en.word_index.items()))
detokenizer_ja = dict(map(reversed, tokenizer_ja.word_index.items()))

In [2]:
#回答確認用
word="好き な 食べ物 は ？" #分かち書き後の入力
word1="<s> " + word.strip() + " </s>"
tmp=[[tokenizer_en.texts_to_sequences([i])[0][0] for i in word1.split()]]
input_seq =pad_sequences(tmp, 26, padding='post')
bos_eos = tokenizer_ja.texts_to_sequences(["<s>", "</s>"])
output_seq, attention_seq = decode_sequence(input_seq, bos_eos)
print("発話文:「",word,"」")
print('回答文:「', ' '.join([detokenizer_ja[i] for i in output_seq[1:-1]]),"」")

NameError: name 'tokenizer_en' is not defined